# **Load libraries/packages**

In [ ]:
import scvi
import scanpy
import pandas
import glob
import os
import warnings
from time import gmtime, strftime
from sinfo import sinfo

In [ ]:
warnings.filterwarnings('ignore')

## Start Time

In [ ]:
strftime("%Y-%m-%d %H:%M:%S", gmtime())

## Load Params

In [ ]:
# %run -i 'param_files/scrublet_params.py'

In [ ]:
proj_dir = os.path.join(os.getcwd(),'../')
proj_dir

In [ ]:
alevin_dir = f'{proj_dir}analyses/doublets/filtered_inputs/'
alevin_dir

In [ ]:
output_dir = os.path.join(proj_dir,'analyses/doublets/solo/')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
output_dir

### List input directories

In [ ]:
os.chdir(output_dir)
myList = glob.glob(f'{alevin_dir}*')
myList

## Run Solo

In [ ]:
for my_dir_path in myList:

    #dir_name = my_dir_path[78:]
    dir_name = my_dir_path.split('/')[-1]
    print('\n\n\n\nProcessing: ',my_dir_path,'\n\n')

    # if not os.path.exists(output_dir + dir_name):
    #     os.makedirs(output_dir + dir_name)

    print('\n\n')
    print('Load the data ...')
    # Load the data
    adata = scanpy.read_10x_mtx(my_dir_path)
    adata

    print('\n\n')
    print('Setup SCVI model ...')
    # Setup SCVI model
    scvi.model.SCVI.setup_anndata(adata)
    vae = scvi.model.SCVI(adata)
    vae.train()

    print('\n\n')
    print('Solo: Train and Predict ...')
    # Solo: Train and Predict
    solo = scvi.external.SOLO.from_scvi_model(vae)
    solo.train()
    my_scores = solo.predict(soft = True)
    my_predictions = solo.predict(soft = False)
    my_predictions = pandas.DataFrame(my_predictions, columns = ['prediction'])

    print('\n\n')
    print('Save the data ...')
    # Save the data
    # my_prefix = f'{strftime("%Y-%m-%d", gmtime())}_{dir_name}_'
    my_prefix = f'{dir_name}_'
    solo.save(dir_path = output_dir, prefix = my_prefix, overwrite = True, save_anndata = True)
    my_scores.to_csv(f'{output_dir}/{my_prefix}solo_doublet_scores.csv', index = True, header = True, sep = ',')
    my_predictions.to_csv(f'{output_dir}/{my_prefix}solo_doublet_predictions.csv', index = True, header = True, sep = ',')


## Session Info

In [ ]:
sinfo()

## Stop time

In [ ]:
strftime("%Y-%m-%d %H:%M:%S", gmtime())